In [1]:
import numpy as np
import pandas as pd
import os
import cv2

In [2]:
train=pd.read_csv('/home/admin238/Downloads/age-detection/train.csv')
train.head()

,ID,Class
0,377.jpg,MIDDLE
1,17814.jpg,YOUNG
2,21283.jpg,MIDDLE
3,16496.jpg,YOUNG
4,4487.jpg,MIDDLE


In [3]:
test=pd.read_csv('/home/admin238/Downloads/age-detection/test.csv')
test.head()

,ID
0,25321.jpg
1,989.jpg
2,19277.jpg
3,13093.jpg
4,5367.jpg


In [4]:
train.describe()


,ID,Class
count,19906,19906
unique,19906,3
top,7563.jpg,MIDDLE
freq,1,10804


In [5]:
def image_read(imglist,path):
    image_array=[]
    for i in imglist:
        image=cv2.imread(path+i)
        image=cv2.resize(image,(64,64))
        image_array.append(image)
    image_array=np.array(image_array)    
    return image_array

In [6]:
train_img_path="Train/"
imglist=train['ID']
#print(imglist)
X_train=image_read(imglist,train_img_path)


In [7]:
X_train.shape


(19906, 64, 64, 3)

In [8]:
test_img_path="Test/"
imglist1=test['ID']
X_test=image_read(imglist1,test_img_path)

In [9]:
X_test.shape


(6636, 64, 64, 3)

In [10]:
X_train=X_train/255

In [11]:
X_test=X_test/255


In [12]:
Y_train=train['Class']
Y_train = Y_train.map({'YOUNG': 0, 'MIDDLE': 1, 'OLD': 2})

In [13]:
from sklearn.preprocessing import LabelBinarizer 
label_binarizer = LabelBinarizer()
label_binarizer.fit(Y_train)

Y_train=label_binarizer.transform(Y_train)

In [14]:
from sklearn.model_selection import train_test_split
X_train, X_valid, Y_train, Y_valid=train_test_split(X_train,Y_train,test_size=0.15, random_state=42)

In [15]:
X_train.shape

(16920, 64, 64, 3)

In [16]:
from keras.preprocessing.image import ImageDataGenerator


Using TensorFlow backend.


In [17]:
datagen_train = ImageDataGenerator(
    width_shift_range=0.2,  # randomly shift images horizontally 
    height_shift_range=0.2,# randomly shift images vertically 
    
    horizontal_flip=True) # randomly flip images horizontally

# fit augmented image generator on data
datagen_train.fit(X_train)

In [18]:
from keras.layers import Input,InputLayer, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout
from keras.models import Sequential,Model
from keras.callbacks import ModelCheckpoint

In [19]:
inputShape=(64,64,3)

In [20]:
input = Input(inputShape)
#x = Conv2D(32,(3,3),strides = (1,1),name='conv_layer1')(xInput)

x = Conv2D(64,(3,3),strides = (1,1),name='layer_conv1',padding='same')(input)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = MaxPooling2D((2,2),name='maxPool1')(x)
#x = Dropout(0.5)(x)

x = Conv2D(128,(3,3),strides = (1,1),name='layer_conv2')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = MaxPooling2D((2,2),name='maxPool2')(x)
#x = Dropout(0.5)(x)

x = Conv2D(128,(3,3),strides = (1,1),name='layer_conv3')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = MaxPooling2D((2,2),name='maxPool3')(x)
#x = Dropout(0.5)(x)

x = Conv2D(64,(3,3),strides = (1,1),name='conv4')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = MaxPooling2D((2,2),name='maxPool4')(x)


x = Flatten()(x)
x = Dense(64,activation = 'relu',name='fc0')(x)
x = Dropout(0.25)(x)
x = Dense(32,activation = 'relu',name='fc1')(x)
x = Dropout(0.25)(x)
x = Dense(3,activation = 'softmax',name='fc2')(x)

model = Model(inputs = input,outputs = x,name='Predict')


W0726 11:38:56.235521 140045543417664 deprecation_wrapper.py:118] From /home/admin238/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0726 11:38:58.040969 140045543417664 deprecation_wrapper.py:118] From /home/admin238/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:1834: The name tf.nn.fused_batch_norm is deprecated. Please use tf.compat.v1.nn.fused_batch_norm instead.

W0726 11:38:58.385651 140045543417664 deprecation.py:506] From /home/admin238/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [21]:
model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 64, 64, 3)         0         
_________________________________________________________________
layer_conv1 (Conv2D)         (None, 64, 64, 64)        1792      
_________________________________________________________________
batch_normalization_1 (Batch (None, 64, 64, 64)        256       
_________________________________________________________________
activation_1 (Activation)    (None, 64, 64, 64)        0         
_________________________________________________________________
maxPool1 (MaxPooling2D)      (None, 32, 32, 64)        0         
_________________________________________________________________
layer_conv2 (Conv2D)         (None, 30, 30, 128)       73856     
_________________________________________________________________
batch_normalization_2 (Batch (None, 30, 30, 128)       512       
__________

In [22]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


W0726 11:38:58.614203 140045543417664 deprecation_wrapper.py:118] From /home/admin238/anaconda3/lib/python3.7/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [23]:
checkpointer = ModelCheckpoint(filepath='cnnweights.best.eda.hdf5', verbose=1, save_best_only=True)


In [24]:
model.fit_generator(datagen_train.flow(X_train, Y_train, batch_size=16), validation_data=(X_valid, Y_valid),
                          epochs=3,steps_per_epoch=X_train.shape[0],callbacks=[checkpointer], verbose=1)

W0726 11:38:59.274007 140045543417664 deprecation.py:323] From /home/admin238/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/ops/math_grad.py:1251: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/5
16920/16920 [==============================] - 2478s 146ms/step - loss: 0.7049 - acc: 0.6942 - val_loss: 0.6322 - val_acc: 0.7244

Epoch 00001: val_loss improved from inf to 0.63224, saving model to cnnweights.best.eda.hdf5
Epoch 2/5
16920/16920 [==============================] - 2508s 148ms/step - loss: 0.5385 - acc: 0.7782 - val_loss: 0.4612 - val_acc: 0.8068

Epoch 00002: val_loss improved from 0.63224 to 0.46119, saving model to cnnweights.best.eda.hdf5
Epoch 3/5
16920/16920 [==============================] - 2489s 147ms/step - loss: 0.4507 - acc: 0.8200 - val_loss: 0.4286 - val_acc: 0.8272

Epoch 00003: val_loss improved from 0.46119 to 0.42863, saving model to cnnweights.best.eda.hdf5
Epoch 4/5
16920/16920 [==============================] - 2486s 147ms/step - loss: 0.3921 - acc: 0.8462 - val_loss: 0.5136 - val_acc: 0.7894

Epoch 00004: val_loss did not improve from 0.42863
Epoch 5/5
16920/16920 [==============================] - 2574s 152ms/step - loss: 0.3493 - acc: 0.

In [25]:
X_test.shape

(6636, 64, 64, 3)

In [26]:
pred=model.evaluate(X_train,Y_train)
print("Accuracy : " +str(pred[1]*100))
print("Total Loss  " +str(pred[0]*100))

16920/16920 [==============================] - 42s 3ms/step
Accuracy : 88.72340425813734
Total Loss  28.690681131314445


In [27]:
print("on valid data")
pred1=model.evaluate(X_valid,Y_valid)
print("accuaracy", str(pred1[1]*100))
print("Total loss",str(pred1[0]*100))


on valid data
2986/2986 [==============================] - 8s 3ms/step
accuaracy 83.48961821036777
Total loss 42.796172087853655


In [28]:
model.save_weights('final_weight_cnn.h5')

In [29]:
predictions = model.predict(X_test)
predictions = np.argmax(predictions, axis= 1)

In [30]:
classes={0: 'YOUNG', 1: 'MIDDLE', 2: 'OLD'}


In [31]:
predicted_class=[classes[x] for x in predictions]


In [32]:
predicted_class[0:3]


['YOUNG', 'YOUNG', 'YOUNG']

In [33]:
sub = pd.DataFrame({
    "Class": predicted_class,
    "ID": test['ID']
})

In [34]:
sub.to_csv("cnn_predicted_class.csv", index=False)
